### Links:
https://pythoninoffice.com/get-outlook-calendar-meeting-data-using-python/ <br>
restrict method: https://learn.microsoft.com/es-es/office/vba/api/Outlook.Items.Restrict

### Info:
* cal[1].subject gives the meeting name of the 2nd meeting in that list.
* cal[1].start gives the starting time for the same meeting.
* cal[1].end gives the ending time for the same meeting.
* cal[1].body contains the content (body) of the meeting invite.

In [26]:
import datetime as dt
import pandas as pd
import win32com.client

In [27]:
def get_calendar(begin,end):
    outlook = win32com.client.Dispatch('Outlook.Application').GetNamespace('MAPI')
    calendar = outlook.getDefaultFolder(9).Items
    calendar.IncludeRecurrences = True
    calendar.Sort('[Start]')
    begin = begin.strftime('%Y/%m/%d')
    end = end.strftime('%Y/%m/%d')
    restriction = f"[Start] >= '{begin}' And [End] <= '{end}'"
    calendar = calendar.Restrict(restriction)
    return calendar

In [28]:
# Año, día, mes
begin = dt.datetime(2024,4,1)
end = dt.datetime(2024,8,12)
cal = get_calendar(begin, end)

In [29]:
calendario = pd.DataFrame()
calendario['CATEGORIA'] = [app.categories for app in cal]
calendario['EVENTO'] = [app.subject for app in cal]
calendario['F_INICIO'] = [app.start.date() for app in cal]
calendario['T_INICIO'] = [app.start.time() for app in cal]
calendario['DURACION_min'] = [(app.end - app.start).total_seconds()/60 for app in cal]
calendario['CONTENIDO'] = [app.body for app in cal]
calendario = calendario[calendario['F_INICIO']<= end.date()]
calendario = calendario[(calendario['CATEGORIA'] != 'Personal') &
                        (calendario['CATEGORIA'] != 'Holiday')]

In [30]:
# Exporta la información a un excel.
nombre_archivo = 'diciembre_2023'
calendario.sort_values(by='F_INICIO',ascending=False).to_excel(f'{nombre_archivo}.xlsx',index = False)

In [31]:
calendario.shape

(192, 6)

In [32]:
resumen = calendario[['CATEGORIA','DURACION_min']].groupby(['CATEGORIA']).sum()
resumen['DURACION_h'] = round(resumen['DURACION_min'] / 60,2)
resumen['DURACION_d'] = round(resumen['DURACION_h'] / 24,2)
# print(f"Total de horas semanales: {round(resumen['DURACION_h'].sum())}")
# print(f"Porcentage ocupación: {round(resumen['DURACION_h'].sum() / (5*8/365) ,2)}")
resumen.sort_values(['DURACION_h'],ascending=False)

,DURACION_min,DURACION_h,DURACION_d
CATEGORIA,,,
FOE - COE,4440.0,74.00,3.08
Proyectos,3420.0,57.00,2.38
Capacitaciones,2255.0,37.58,1.57
PCP,1620.0,27.00,1.12
BES,1470.0,24.50,1.02
Administrativo,1125.0,18.75,0.78
,600.0,10.00,0.42
Información,480.0,8.00,0.33
PCP; RIFTs,270.0,4.50,0.19


In [33]:
calendario

,CATEGORIA,EVENTO,F_INICIO,T_INICIO,DURACION_min,CONTENIDO
0,FOE - COE,Novedades de equipos y plan de trabajo semanal,2024-01-08,08:30:00,120.0,Quedan invitados a participar de la reunión de...
1,FOE - COE,Novedades de equipos y plan de trabajo semanal,2024-01-15,08:30:00,120.0,Quedan invitados a participar de la reunión de...
2,FOE - COE,Novedades de equipos y plan de trabajo semanal,2024-01-22,08:30:00,120.0,Quedan invitados a participar de la reunión de...
3,FOE - COE,Novedades de equipos y plan de trabajo semanal,2024-01-29,08:30:00,120.0,Quedan invitados a participar de la reunión de...
4,FOE - COE,Novedades de equipos y plan de trabajo semanal,2024-02-05,08:30:00,120.0,Quedan invitados a participar de la reunión de...
...,...,...,...,...,...,...
191,Planificación,Líneas de trabajo y objetivos semana siguiente,2024-08-09,14:30:00,60.0,Actualizar planilla Seguimiento_Proyectos_Pen...
192,BES,Puesta en valor de la estandarización de BES,2024-08-09,15:30:00,90.0,Martin:\r\n\r\nDejo agendado este espacio para...
193,FOE - COE,Novedades de equipos y plan de trabajo semanal,2024-08-12,08:30:00,120.0,Quedan invitados a participar de la reunión de...
194,FOE - COE,P° óptima de inyección en CD,2024-08-12,11:00:00,60.0,Muchachos\r\n\r\n \r\n\r\nTenemos que analizar...


In [34]:
# Proyectos
rubro = 'Proyectos'
df_rubro = calendario[calendario.CATEGORIA == rubro]

In [35]:
descripcion = list(df_rubro.CONTENIDO)
print(descripcion[2])

PARTE 2
Modeling Flow Profile Using Distributed Temperature Sensor (DTS) System <file:///C:/Users/xixz02/Desktop/IZ/2.-%20Ingeniería/2.-%20Proyectos/2.-%20Temp_Flow/1.-%20Teoría/spe-111790-ms.pdf>  
Video presentación de OnePetro


In [38]:
# for i in descripcion:
#     print(i)

In [39]:
calendario[calendario.EVENTO.str.contains('RIFT')]

,CATEGORIA,EVENTO,F_INICIO,T_INICIO,DURACION_min,CONTENIDO
49,Capacitaciones,RV: ESP-RIFTS Reliability Training Session,2024-05-08,11:00:00,120.0,\r\n\r\n \r\n\r\n--------Cita original-------...
52,Capacitaciones,RV: ESP-RIFTS Reliability Training Session,2024-05-09,11:00:00,120.0,\r\n\r\n \r\n\r\n--------Cita original-------...
80,PCP; RIFTs,Reunión ESP RIFTS Mayo 2024,2024-05-31,11:00:00,60.0,Buen día. Dejo reservado el espacio para mostr...
161,PCP; RIFTs,PCP RIFT,2024-07-24,09:30:00,150.0,Tareas:\r\n*\tAnálisis de la información reque...
163,PCP; RIFTs,RIFT - PCP,2024-07-25,10:00:00,60.0,Agendo el espacio para ver la data del Rift.\r...
